In [1]:
import pandas as pd

import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Pusher 
from tfx.proto import example_gen_pb2, trainer_pb2, pusher_pb2 
from tfx.v1.dsl import Importer
from tfx.v1.types.standard_artifacts import HyperParameters
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

In [2]:
PIPELANE_NAME = "marketing-classification-pipeline"

# Pipeline inputs
DATA_ROOT = "data"
TRANSFORM_MODULE_FILE = "modules/transform.py"
TUNER_MODULE_FILE = "modules/tuner.py"
TRAINER_MODULE_FILE = "modules/trainer.py"

# Pipeline outputs
OUTPUT_BASE = "outputs"

SERVING_MODEL_DIR = os.path.join(OUTPUT_BASE, "serving_model")
PIPELINE_ROOT = os.path.join(OUTPUT_BASE, PIPELANE_NAME)
METADATA_PATH = os.path.join(PIPELINE_ROOT, "metadata.sqlite")

In [3]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [4]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 131
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [5]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"],
)
 
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 132
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [6]:
interactive_context.show(statistics_gen.outputs["statistics"])

In [7]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 133
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [8]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'campaign',INT,required,,-
'cons.conf.idx',FLOAT,required,,-
'cons.price.idx',FLOAT,required,,-
'contact',STRING,required,,'contact'
'day_of_week',STRING,required,,'day_of_week'
'default',STRING,required,,'default'
'education',STRING,required,,'education'
'emp.var.rate',FLOAT,required,,-


,Values
Domain,
'contact',"'cellular', 'telephone'"
'day_of_week',"'fri', 'mon', 'thu', 'tue', 'wed'"
'default',"'no', 'unknown', 'yes'"
'education',"'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 'illiterate', 'professional.course', 'university.degree', 'unknown'"
'housing',"'no', 'unknown', 'yes'"
'job',"'admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown'"
'loan',"'no', 'unknown', 'yes'"
'marital',"'divorced', 'married', 'single', 'unknown'"
'month',"'apr', 'aug', 'dec', 'jul', 'jun', 'mar', 'may', 'nov', 'oct', 'sep'"


In [9]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 134
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [10]:
interactive_context.show(example_validator.outputs['anomalies'])

In [11]:
TRANSFORM_MODULE_FILE = "modules/transform.py"

In [45]:
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: outputs\marketing-classification-pipeline\Transform\transform_graph\157\.temp_path\tftransform_tmp\7f77638917e64df296eb0e6ab42ad990\assets


INFO:tensorflow:Assets written to: outputs\marketing-classification-pipeline\Transform\transform_graph\157\.temp_path\tftransform_tmp\7f77638917e64df296eb0e6ab42ad990\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 157
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [78]:
param_loc = "outputs/marketing-classification-pipeline/Tuner/best_hyperparameters/95"

In [79]:
hparams_importer =  Importer(
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (keras_tuner.HyperParameters.get_config())
    source_uri=os.path.abspath(param_loc),
    artifact_type=HyperParameters,
).with_id('import_hparams')

interactive_context.run(hparams_importer)

ExecutionResult(
    component_id: import_hparams
    execution_id: 171
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 125
        type_id: 30
        uri: "c:\\Users\\ACER\\Documents\\USU\\Semester6\\DBS_Journey\\mlops_2\\outputs\\marketing-classification-pipeline\\Tuner\\best_hyperparameters\\95"
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.10.0"
          }
        }
        create_time_since_epoch: 1750815289028
        last_update_time_since_epoch: 1750815290288
        , artifact_type: id: 30
        name: "HyperParameters"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [85]:
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs["schema"],
    hyperparameters=hparams_importer.outputs['result'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)
interactive_context.run(trainer)

Features: {'age_xf': <tf.Tensor: shape=(64, 1), dtype=float32, numpy=
array([[ 1.6230052 ],
       [ 1.527289  ],
       [ 0.85727644],
       [-0.29131654],
       [-0.0041683 ],
       [ 0.18726386],
       [-0.9613291 ],
       [-0.09988438],
       [-0.09988438],
       [ 1.1444247 ],
       [ 0.4744121 ],
       [-0.9613291 ],
       [-0.86561304],
       [-1.1527613 ],
       [ 1.4315729 ],
       [-0.5784648 ],
       [-1.1527613 ],
       [-0.769897  ],
       [-0.0041683 ],
       [-0.0041683 ],
       [ 0.85727644],
       [ 0.4744121 ],
       [ 0.85727644],
       [-0.0041683 ],
       [ 1.527289  ],
       [-0.769897  ],
       [ 0.09154778],
       [-0.67418087],
       [ 1.2401408 ],
       [ 0.09154778],
       [ 0.09154778],
       [ 0.18726386],
       [ 0.5701282 ],
       [ 0.5701282 ],
       [-1.4399096 ],
       [-0.86561304],
       [ 1.4315729 ],
       [ 0.9529925 ],
       [ 0.09154778],
       [-0.9613291 ],
       [-0.48274872],
       [-0.48274872],
      

INFO:tensorflow:Assets written to: outputs\marketing-classification-pipeline\Trainer\model\175\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\marketing-classification-pipeline\Trainer\model\175\Format-Serving\assets


5153/5153 [==============================] - 56s 11ms/step - loss: 0.2952 - binary_accuracy: 0.8892 - val_loss: 0.2804 - val_binary_accuracy: 0.8862
Epoch 2/10
5151/5153 [============================>.] - ETA: 0s - loss: 0.2923 - binary_accuracy: 0.8875
Epoch 2: val_binary_accuracy did not improve from 0.88617
5153/5153 [==============================] - 49s 10ms/step - loss: 0.2923 - binary_accuracy: 0.8875 - val_loss: 0.2840 - val_binary_accuracy: 0.8862
Epoch 3/10
5149/5153 [============================>.] - ETA: 0s - loss: 0.2998 - binary_accuracy: 0.8878
Epoch 3: val_binary_accuracy did not improve from 0.88617
5153/5153 [==============================] - 47s 9ms/step - loss: 0.2999 - binary_accuracy: 0.8877 - val_loss: 0.3129 - val_binary_accuracy: 0.8801
Epoch 4/10
5152/5153 [============================>.] - ETA: 0s - loss: 0.2939 - binary_accuracy: 0.8876
Epoch 4: val_binary_accuracy did not improve from 0.88617
5153/5153 [==============================] - 57s 11ms/step - loss

INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: outputs\marketing-classification-pipeline\Trainer\model\175\Format-Serving\assets


INFO:tensorflow:Assets written to: outputs\marketing-classification-pipeline\Trainer\model\175\Format-Serving\assets


You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


ExecutionResult(
    component_id: Trainer
    execution_id: 175
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [86]:
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
 
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 176
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

In [89]:
import tensorflow_model_analysis as tfma 
 
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(
        label_key='y_xf',
    )],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
)

In [91]:
from tfx.components import Evaluator
evaluator = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 179
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [92]:
# Visualize the evaluation results
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [93]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR))
 
)
 
interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 180
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))